<img src="http://oci02.img.iteso.mx/identidad_de_instancia_2018/ITESO/Logos%20ITESO/Logo-ITESO-Principal.jpg">

# Microestructuras y Sistemas de Traiding 

## Autores: Ariadna Galindo y Bruno Pimentel 
### Otoño 2021

# 0.Librerias y dependencias 

## 0.1 Librerias necesarias

La lista completa requirements se encuentra en *requirements.txt* 

* certifi==2021.5.30
* charset-normalizer==2.0.4
* idna==3.2
* lxml==4.6.3
* mpmath==1.2.1
* numpy==1.21.2
* pandas==1.3.2
* pandas-datareader==0.10.0
* python-dateutil==2.8.2
* pytz==2021.1
* requests==2.26.0
* six==1.16.0
* sympy==1.8
* urllib3==1.26.6

In [12]:
#importamos librerias 
import math as m
import matplotlib.pyplot as plt
import pandas_datareader.data as web
import pandas as pd
import os 
import sys 
import glob

## 0.2 Dependencias 
Para correr correctamente este notebook se requiere tener la carpeta /files con los archivos de NAFTRAC tal y como la incluimos en el repositorio. Adicionalmente se recomienda correr primero los siguientes archivos en el siguiente orden: functions.py>data.py>main.py

# 1. Introducción 
(2 cuartillas )

#### Laboratorio 1: Inversión de Capital

**Caso:** Tienes 1 Millón de pesos, tienes dos opciones para invertirlos. Por un lado, puedes hacer una Inversión Pasiva del capital, comprando el 100% del capital de títulos de un ETF que replica al mercado de renta variable mexicano S&P/BMV IPC que es el NAFTRAC. Pero por otro lado, puedes hacer una Inversión Activa del capital, al crear un portafolio que tenga los mismos activos que el ETF y haciendo el rebalanceo por medio de un algoritmo en específico.

### ¿Qué estrategia de inversión propondrías si tu trabajo fuera invertir 1 Millón de pesos?

# 2. Objetivos 
## 2.1 General

Crear estrategias para invertir 1 millón de pesos utilizando NAFTRAC 

## 2.2 Específicos
* Generar una inversión pasiva 
* Generar una inversión activa



# 3. Datos utilizados 

In [14]:
import data as dt

C:\ProgramData\Anaconda3\lib\site-packages


IndexError: list index out of range

Inversión pasiva

In [2]:
dt.precios_mensuales.head(10)

NameError: name 'dt' is not defined

Inversión activa

In [9]:
# importamos los datos diarios que descargamos de Yahoo! 
dt.precios_diarios.head(10)

NameError: name 'dt' is not defined

# 4. Procesos 

En esta sección se mostrarán brevemente los procesos y funciones empleadas para obtener los resultados. 

Inversión pasiva 

In [18]:
# importamos .py
import functions
import visualizations as vs
import main as main
from data import dataframes, precios_mensuales, capital, comision

C:\ProgramData\Anaconda3\lib\site-packages


IndexError: list index out of range

Recordamos el capital a invertir

In [ ]:
print(capital)

Usamos NAFTRAC al 31/01/2018 como referencia para obtener los pesos de cada activo

In [ ]:
#peso%/100
main.weigths.head(10)

Teniendo los pesos los vamos a multiplicar por el capital para obtener lo que vamos a destinar a cada activo

In [ ]:
ponderaciones=main.weigths*capital
ponderaciones[:10]

Usando los precios NAFTRAC al 31/01/2018 vemos cuantos titulos se obtienen de cada activo, esto lo hacemos dividiendo las ponderaciones entre el precio. 

In [ ]:
precio=precios_mensuales.iloc[0,:]

titulos=[ponderaciones[i]/precio[i] for i in range(0, len(ponderaciones))]
titulos=[m.trunc(titulos[i]) for i in range(0, len(ponderaciones))]

titulos[:10]

Calculamos las comisiones a pagar al comprar los títulos anteriormente mensionados

In [ ]:
comisiones=sum(titulos*precio*comision)
comisiones

Calculamos el CASH que quedó disponible después de comisiones y le sumamos el cash

In [ ]:
cash=capital- sum(titulos*precio)- comisiones
cash

Calculamos el valor del portafolio inicial 

In [ ]:
port_ini= sum(titulos*precio)+ cash
port_ini

Obtenemos el valor de todos los portafolios al final de cada mes

In [ ]:
port_fin=[sum(precios_mensuales.iloc[i,:]*titulos) for i in range(1, len(precios_mensuales))]
port_fin.insert(0, port_ini)

port_fin[:10]

Mostramos como evolucionan los rendimientos y rendimientos acumulados con el paso de los meses

In [ ]:
vs.df.head(10)